In [13]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

In [14]:
spark = SparkSession.builder.appName('ml-fraud').getOrCreate()
df = spark.read.csv('creditcard.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

In [15]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

0         1           2           3          4
Time      0.000000  0.000000    1.000000    1.000000   2.000000
V1       -1.359807  1.191857   -1.358354   -0.966272  -1.158233
V2       -0.072781  0.266151   -1.340163   -0.185226   0.877737
V3        2.536347  0.166480    1.773209    1.792993   1.548718
V4        1.378155  0.448154    0.379780   -0.863291   0.403034
V5       -0.338321  0.060018   -0.503198   -0.010309  -0.407193
V6        0.462388 -0.082361    1.800499    1.247203   0.095921
V7        0.239599 -0.078803    0.791461    0.237609   0.592941
V8        0.098698  0.085102    0.247676    0.377436  -0.270533
V9        0.363787 -0.255425   -1.514654   -1.387024   0.817739
V10       0.090794 -0.166974    0.207643   -0.054952   0.753074
V11      -0.551600  1.612727    0.624501   -0.226487  -0.822843
V12      -0.617801  1.065235    0.066084    0.178228   0.538196
V13      -0.991390  0.489095    0.717293    0.507757   1.345852
V14      -0.311169 -0.143772   -0.165946   -0.287924  -1.119670
V15       1.468177  0.635558    2.345865   -0.631418   0.175121
V16      -0.470401  0.463917   -2.890083   -1.059647  -0.451449
V17       0.207971 -0.114805    1.109969   -0.684093  -0.237033
V18       0.025791 -0.183361   -0.121359    1.965775  -0.038195
V19       0.403993 -0.145783   -2.261857   -1.232622   0.803487
V20       0.251412 -0.069083    0.524980   -0.208038   0.408542
V21      -0.018307 -0.225775    0.247998   -0.108300  -0.009431
V22       0.277838 -0.638672    0.771679    0.005274   0.798278
V23      -0.110474  0.101288    0.909412   -0.190321  -0.137458
V24       0.066928 -0.339846   -0.689281   -1.175575   0.141267
V25       0.128539  0.167170   -0.327642    0.647376  -0.206010
V26      -0.189115  0.125895   -0.139097   -0.221929   0.502292
V27       0.133558 -0.008983   -0.055353    0.062723   0.219422
V28      -0.021053  0.014724   -0.059752    0.061458   0.215153
Amount  149.620000  2.690000  378.660000  123.500000  69.990000
Class     0.000000  0.000000    0.000000    0.000000   0.000000

In [16]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'double']
df.select(numeric_features).describe().toPandas().transpose()

0                        1                    2  \
summary   count                     mean               stddev   
Time     284807        94813.85957508067   47488.145954566324   
V1       284807    2.235360406313924E-15   1.9586958038574904   
V2       284807    6.865749819392767E-17   1.6513085794769997   
V3       284807   -5.824710544452282E-15   1.5162550051777732   
V4       284807   2.0118243656825316E-15    1.415868574940927   
V5       284807    3.704311530463074E-15    1.380246734031437   
V6       284807   1.1400338072201013E-15   1.3322710897575714   
V7       284807  -1.1496139232471609E-16   1.2370935981826632   
V8       284807     -2.9538691083434E-16   1.1943529026692048   
V9       284807   -2.082078549880969E-15   1.0986320892243222   
V10      284807    2.145945990061367E-15   1.0888497654025215   
V11      284807  -1.2773488036079566E-17   1.0207130277115581   
V12      284807  -3.3083334013446075E-15   0.9992013895301415   
V13      284807    1.034652530922445E-15   0.9952742301251558   
V14      284807   1.7084540248256419E-15   0.9585956112570617   
V15      284807    5.933285192758958E-15   0.9153160116104389   
V16      284807   1.3947052249394376E-15   0.8762528873883704   
V17      284807   -2.874034808117902E-17     0.84933706367439   
V18      284807   1.7499678609429005E-15   0.8381762095288453   
V19      284807     9.38053027649593E-16   0.8140405007685797   
V20      284807     3.81607955077877E-16   0.7709250248871159   
V21      284807   2.8500845180502533E-16   0.7345240143713125   
V22      284807   -7.664092821647739E-16   0.7257015604409107   
V23      284807      2.9538691083434E-16   0.6244602955949898   
V24      284807   4.4549535383333125E-15   0.6056470678271603   
V25      284807    1.034652530922445E-15   0.5212780705409427   
V26      284807   1.8082469001075134E-15  0.48222701326105666   
V27      284807  -3.5236864262028866E-16   0.4036324949650313   
V28      284807  -1.1336470632020615E-16  0.33008326416025036   
Amount   284807         88.3496192509521    250.1201092401885   

                         3                 4  
summary                min               max  
Time                   0.0          172792.0  
V1        -56.407509631329  2.45492999121121  
V2       -72.7157275629303  22.0577289904909  
V3       -48.3255893623954  9.38255843282114  
V4       -5.68317119816995  16.8753440335975  
V5       -113.743306711146  34.8016658766686  
V6       -26.1605059358433  73.3016255459646  
V7       -43.5572415712451  120.589493945238  
V8       -73.2167184552674  20.0072083651213  
V9       -13.4340663182301  15.5949946071278  
V10      -24.5882624372475  23.7451361206545  
V11      -4.79747346479757  12.0189131816199  
V12      -18.6837146333443   7.8483920756446  
V13      -5.79188120632084  7.12688295859376  
V14      -19.2143254902614  10.5267660517847  
V15      -4.49894467676621  8.87774159774277  
V16      -14.1298545174931  17.3151115176278  
V17      -25.1627993693248  9.25352625047285  
V18      -9.49874592104677  5.04106918541184  
V19      -7.21352743017759  5.59197142733558  
V20       -54.497720494566  39.4209042482199  
V21      -34.8303821448146  27.2028391573154  
V22       -10.933143697655  10.5030900899454  
V23      -44.8077352037913  22.5284116897749  
V24      -2.83662691870341  4.58454913689817  
V25      -10.2953970749851  7.51958867870916  
V26      -2.60455055280817   3.5173456116238  
V27      -22.5656793207827  31.6121981061363  
V28      -15.4300839055349  33.8478078188831  
Amount                 0.0          25691.16

In [17]:
numericCols = ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26','V27', 'V28', 'Amount']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)
df.show()

+----+------------------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------+-----+--------------------+
|Time|                V1|                 V2|                V3|                 V4|                 V5|                 V6|                  V7|                 V8|                V9|                V10|               V11|               V12|                V13|                V14|                V15|                V16|                 V17|                V18|                V19|                V

In [18]:
label_stringIdx = StringIndexer(inputCol = 'Class', outputCol = 'labelIndex')
df = label_stringIdx.fit(df).transform(df)
df.show()

+----+------------------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------+-----+--------------------+----------+
|Time|                V1|                 V2|                V3|                 V4|                 V5|                 V6|                  V7|                 V8|                V9|                V10|               V11|               V12|                V13|                V14|                V15|                V16|                 V17|                V18|                V19|      

In [19]:
pd.DataFrame(df.take(110), columns=df.columns).transpose()

0    \
Time                                                      0.0   
V1                                                  -1.359807   
V2                                                  -0.072781   
V3                                                   2.536347   
V4                                                   1.378155   
V5                                                  -0.338321   
V6                                                   0.462388   
V7                                                   0.239599   
V8                                                   0.098698   
V9                                                   0.363787   
V10                                                  0.090794   
V11                                                   -0.5516   
V12                                                 -0.617801   
V13                                                  -0.99139   
V14                                                 -0.311169   
V15                                                  1.468177   
V16                                                 -0.470401   
V17                                                  0.207971   
V18                                                  0.025791   
V19                                                  0.403993   
V20                                                  0.251412   
V21                                                 -0.018307   
V22                                                  0.277838   
V23                                                 -0.110474   
V24                                                  0.066928   
V25                                                  0.128539   
V26                                                 -0.189115   
V27                                                  0.133558   
V28                                                 -0.021053   
Amount                                                 149.62   
Class                                                       0   
features    [0.0, -1.3598071336738, -0.0727811733098497, 2...   
labelIndex                                                0.0   

                                                          1    \
Time                                                      0.0   
V1                                                   1.191857   
V2                                                   0.266151   
V3                                                    0.16648   
V4                                                   0.448154   
V5                                                   0.060018   
V6                                                  -0.082361   
V7                                                  -0.078803   
V8                                                   0.085102   
V9                                                  -0.255425   
V10                                                 -0.166974   
V11                                                  1.612727   
V12                                                  1.065235   
V13                                                  0.489095   
V14                                                 -0.143772   
V15                                                  0.635558   
V16                                                  0.463917   
V17                                                 -0.114805   
V18                                                 -0.183361   
V19                                                 -0.145783   
V20                                                 -0.069083   
V21                                                 -0.225775   
V22                                                 -0.638672   
V23                                                  0.101288   
V24                                                 -0.339846   
V25                                                   0.16717   
V26                                                  0.125895   
V27                        

In [20]:
train, test = df.randomSplit([0.67, 0.33], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 190763
Test Dataset Count: 94044


In [31]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'labelIndex', numTrees=30)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26','V27', 'V28', 'Amount', 'labelIndex', 'rawPrediction', 'prediction', 'probability').show(25)

+----+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------+----------+--------------------+----------+--------------------+
|Time|                V1|                 V2|                 V3|                 V4|                 V5|                V6|                 V7|                 V8|                 V9|               V10|                V11|                V12|                V13|                V14|                V15|                V16|                 V17|               

In [32]:
predictions.select("labelIndex", "prediction").show(50)

+----------+----------+
|labelIndex|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|    

In [33]:
evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.9994945888758183
Test Error = 0.0005054111241816761


In [34]:
preds_and_labels = predictions.select(['prediction','labelIndex']).withColumn('labelIndex', F.col('labelIndex').cast(FloatType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','labelIndex'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

[[9.3871e+04 1.3000e+01]
 [3.3000e+01 1.2700e+02]]
